In [815]:
import random
import numpy as np
import math
import scipy.special
from rich.console import Console
from rich.table import Table

In [816]:
domains = {
    'x1': [i for i in range(0,16)],
    'x2': [i for i in range(0,11)],
    'x3': [i for i in range(0,26)],
    'x4': [i for i in range(0,5)],
    'x5': [i for i in range(0,31)],
}

In [817]:
constraints = {
    ('x1', 'x2'): lambda x1, x2:  164*x1 <= 3800-310*x2,
    ('x2', 'x1'): lambda x2, x1:  3800-310*x2 >= 164*x1,
    ('x3', 'x4'): lambda x3, x4:  46*x3 <= 2800-111*x4,
    ('x4', 'x3'): lambda x4, x3:  2800-111*x4 >= 46*x3,
    ('x3', 'x5'): lambda x3, x5:  46*x3 <= 3500-12*x5,
    ('x5', 'x3'): lambda x5, x3:  3500-12*x5 >= 46*x3,
}

In [818]:
def revise(x, y):
    revised = False
    x_domain = domains[x]
    y_domain = domains[y]
    all_constraints = [
        constraint for constraint in constraints if constraint[0] == x and constraint[1] == y]
    for x_value in x_domain:
        satisfies = False
        for y_value in y_domain:
            for constraint in all_constraints:
                constraint_func = constraints[constraint]
                if constraint_func(x_value, y_value):
                    satisfies = True
        if not satisfies:
            x_domain.remove(x_value)
            revised = True
    return revised

def ac3(arcs):
    queue = arcs[:]
    while queue:
        (x, y) = queue.pop(0)
        revised = revise(x, y)
        if revised:
            neighbors = [neighbor for neighbor in arcs if neighbor[1] == x]
            queue = queue + neighbors

In [819]:
arcs = [
    ('x1', 'x2'), ('x2', 'x1'),
    ('x3', 'x4'), ('x4', 'x3'),
    ('x3', 'x5') ,('x5', 'x3'),
]

In [820]:
ac3(arcs)

In [821]:
def maximizar(x):
    x1,x2,x3,x4,x5=x
    return 67*x1+91*x2+43*x3+71*x4+23*x5

In [822]:
def minimizar(x):
    x1,x2,x3,x4,x5=x
    return 164*x1+310*x2+46*x3+111*x4+12*x5

In [823]:
def restricciones(x):
    x1,x2,x3,x4,x5=x
    return (0<= x1 and x1<=15 and 0<=x2 and x2<=10 and 0<=x3 and x3<=25 and 0<=x4 and x4<=4 and 0<=x5 and x5<=30 \
            and 164*x1+310*x2<=3800 and 46*x3+111*x4<= 2800 and 46*x3+13*x5<= 3500)

In [824]:
def scalarizing(x,wp, wq):
    x1,x2,x3,x4,x5=x
    cap=5000
    frac1=(67*x1+91*x2+43*x3+71*x4+23*x5)/3418
    frac2=(cap-(164*x1+310*x2+46*x3+111*x4+12*x5)) / (cap-0)
    return wp*frac1 + wq*frac2

In [825]:
wp=0.5
wq=0.5

In [826]:
def Cl() -> float:
    return random.uniform(0.674,1.5)

In [827]:
def gliding():
    ro = 1.204#𝜌
    hg = 8
    V = 5.25
    S = 154
    Gc = 1.9
    Cd = 0.60
    L = 1 / 2 * (ro * (Cl()) * (V**2) * S)
    D = 1 / 2 * (ro * (V**2) * S * Cd)
    artc = np.arctan(D / L)
    dg = hg / np.tan(artc)
    sf = 18
    return dg / sf


In [828]:
def rho():
    beta=1.5
    return ((scipy.special.gamma(beta+1)*np.sin((math.pi*beta)/2))/ \
     (scipy.special.gamma((1+beta)/2)*beta*(2)**((beta-1)/2)))**(1/beta)
rho()

0.6965745025576968

In [829]:
def levy():
    beta = 1.5
    sigma = (scipy.special.gamma(1 + beta) * np.sin(np.pi * beta / 2) /
             (scipy.special.gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
    u = np.random.normal(0, sigma, size=len(lower_band))
    v = np.random.normal(0, 1, size=len(lower_band))
    step = u / np.abs(v)**(1 / beta)
    return 0.01 * step

In [830]:
def randomloc(index):
    return lower_band[index] + levy()[index] * (upper_band[index] - lower_band[index])

In [831]:
def actualizar_valores(i, ref_vector, Pdp):
    for u, v in enumerate(i):
        if random.uniform(0, 1) >= Pdp:
            new_value = v + gliding() * 1.9 * (ref_vector[u] - v)
        else:
            new_value = randomloc(u)
        i[u] = new_value
    return np.clip(i, lower_band, upper_band)

In [832]:
def Smin(t,maxiter):
    return (10 * 10**-6)/(365**(t/(maxiter/2.5)))

In [833]:
maxiter=1000
popsize=50
lower_band=[domains[f"x{i+1}"][0] for i in range(len(domains.keys()))]
upper_band=[domains[f"x{i+1}"][-1] for i in range(len(domains.keys()))]
# lower_band
# upper_band

peor fitness = hickory \
los siguientes 3 = bellota a hickory \
el rest = normal

In [834]:
FS=[]
for i in range(popsize):
    while True:
        aux=[random.uniform(lower_band[j], upper_band[j]) for j in range(len(lower_band))]
        if restricciones(aux):
            break
    FS.append(aux)

In [835]:
for i in FS:
    print(i)

[0.5416590274288324, 8.662234616347709, 10.416656336346671, 1.9058372819164036, 9.667461012628]
[6.688369986430106, 1.0532962835185589, 20.006994365069232, 2.8577223759316723, 8.555393409970858]
[3.472723884870823, 8.00172558698023, 21.12182743288706, 2.989269022443677, 6.109718659602241]
[5.585389181306706, 5.895552178962119, 19.455549682189453, 1.8260746749330394, 8.982741641218608]
[1.1328365523819435, 3.1714873250313746, 9.504125639225606, 3.604934555110644, 2.0076035055754526]
[5.726196247312751, 0.7875746441488152, 18.550818145812734, 2.6842014087917345, 3.1724998372063085]
[11.896123272573892, 4.844233215752135, 8.75554449838814, 1.2943831641178045, 10.391451716621686]
[8.015928061229825, 2.28750543549829, 15.816073174796793, 0.9441815374006688, 12.139410985599127]
[1.9345793643841462, 3.9701280100792324, 15.26361585804768, 3.4813531441211416, 21.106484646016753]
[4.015195503582534, 1.3707804620182151, 13.79655726932536, 3.7622505608915247, 21.024134497363598]
[7.874310827945442

In [836]:
fitness_values = [scalarizing(ind,wp,wq) for ind in FS]

In [837]:
fitness_values

[0.3803786953221582,
 0.4875279841926609,
 0.38233985901399337,
 0.40988542910840675,
 0.45426326313282406,
 0.4845885328721676,
 0.37218037466852716,
 0.45897669919260853,
 0.4860095226525012,
 0.5153988021088578,
 0.36722451523701877,
 0.41913326106555254,
 0.5221714394378918,
 0.4034452583234175,
 0.4306326900733082,
 0.4633408975467057,
 0.5334704140376242,
 0.44867039109197526,
 0.39287718222634616,
 0.472671917339319,
 0.38238520933892567,
 0.36169175069351334,
 0.44476210559310875,
 0.40796693886005186,
 0.43324294305717836,
 0.46442156006003693,
 0.4056887258153957,
 0.41860869598769046,
 0.4881801807292272,
 0.4219026274132188,
 0.5094719054580641,
 0.32748631651700183,
 0.3617744119501716,
 0.42320102689190364,
 0.4537969738693892,
 0.41622202103302475,
 0.37922370364416264,
 0.3881265506539989,
 0.3787300044586386,
 0.38323692704277534,
 0.4569331863231231,
 0.3917387751982881,
 0.4576281195124663,
 0.4836412157292282,
 0.46603823947232853,
 0.3880799428659033,
 0.4408757878

In [838]:
FS = [x for _, x in sorted(zip(fitness_values, FS), key=lambda pair: pair[0])]

In [839]:
for i in FS:
    print(i)

[11.435370581936562, 6.007931171722682, 2.2993673239791956, 0.02643181477694645, 2.4712622489211333]
[0.7063310956395125, 9.51558338533588, 0.8429897324567087, 0.5097287009475897, 15.52504583580999]
[6.655643257283948, 7.695818989467408, 13.032186672086732, 0.4942286382628316, 9.306660831700402]
[7.874310827945442, 7.717866969168222, 13.752570432374716, 3.177286016573995, 16.04231217694361]
[11.896123272573892, 4.844233215752135, 8.75554449838814, 1.2943831641178045, 10.391451716621686]
[4.773135201137423, 6.9698846904256415, 9.84948982932429, 1.67633415198314, 8.343818983038414]
[6.857326266790169, 7.401117131956468, 20.455500341689714, 1.341359930873025, 10.057425580883926]
[0.5416590274288324, 8.662234616347709, 10.416656336346671, 1.9058372819164036, 9.667461012628]
[3.472723884870823, 8.00172558698023, 21.12182743288706, 2.989269022443677, 6.109718659602241]
[4.62508429883449, 7.800863481262077, 12.976334045117191, 3.8186344680140603, 14.636513107870861]
[4.531408754075256, 6.5917

In [840]:
trees = [1]
if popsize > 1:
    trees.extend([2] * min(popsize - 1, 3))
if popsize > 4:
    trees.extend(random.randint(3, 4) for _ in range(popsize - 4))

In [841]:
Pdp=0.01
for iterable in range(maxiter):
    Sc = []
    for j, i in enumerate(FS[1:4]):
        resta = 0
        while True:
            FS[j + 1] = actualizar_valores(i, FS[0], Pdp)
            if restricciones(FS[j+1]):
                break
        resta = np.array(i) - np.array(FS[0])
        resta = np.sqrt(np.sum(resta**2))
        Sc.append(resta)

    for idx, j in enumerate(FS[4:]):
        if trees[idx + 4] == 3:
            ref_vector = random.choice(FS[1:4])
        else:
            ref_vector = FS[0]
        while True:
            FS[idx + 4] = actualizar_valores(j, ref_vector, Pdp)
            if restricciones(FS[idx + 4]):
                break
        
    for idx, value in enumerate(Sc):
        if value < Smin(iterable, maxiter):
            while True:
                for i in range(len(FS[idx + 1])):
                    FS[idx + 1][i] = randomloc(i)
                if restricciones(FS[idx + 1]):
                    break
            FS[idx + 1] = np.clip(FS[idx + 1], lower_band, upper_band)

    fitness_values = [scalarizing(ind,wp,wq) for ind in FS]
    FS = [x for _, x in sorted(zip(fitness_values, FS), key=lambda pair: pair[0])]

    trees = [1]
    if popsize > 1:
        trees.extend([2] * min(popsize - 1, 3))
    if popsize > 4:
        trees.extend(random.randint(3, 4) for _ in range(popsize - 4))

In [842]:
cons=Console()
tabla=Table(title="SSA FS resultantes",show_header=True,header_style="bold magenta")
tabla.add_column("FS",justify="full",no_wrap=True)
tabla.add_column("Maximizacion",style="dim",justify="full",no_wrap=True)
tabla.add_column("Minimizacion",style="dim",justify="full")
tabla.add_column("Scalarizing",style="dim",justify="full")
tabla.add_column("Restricciones",style="dim",justify="full")
for j in FS:
    tabla.add_row(str(j),str(maximizar(j)),str(minimizar(j)),str(scalarizing(j,wp,wq)),str(restricciones(j)))

In [843]:
cons.print(tabla)

                                                SSA FS resultantes                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃ FS                                                            ┃ Maximizacion       ┃ Minimi… ┃ Scalar… ┃ Restr… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.9756097560974 │ 4244.0  │ 0.2920… │ True   │
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.9756097560976 │ 4244.0  │ 0.2920… │ True   │
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.9756097560914 │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.975609755548  │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.975609755012  │ 4243.9… │ 0.2920… │ True   │
│ [4.26829268e+00 1.00000000e+01 0.00000000e+00 4.00000000e+00  │ 1479.9756097537477 │ 4243.9… │ 0.2920… │ True   │
│  2.29180953e-13]                                              │                    │         │         │        │
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.975609739761  │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.9756097256918 │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.975609704611  │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26829268 10.          0.          4.          0.        ] │ 1479.9756095804046 │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26829267 10.          0.          4.          0.        ] │ 1479.9756086764567 │ 4243.9… │ 0.2920… │ True   │
│ [4.26829267e+00 1.00000000e+01 0.00000000e+00 4.00000000e+00  │ 1479.9756086415746 │ 4243.9… │ 0.2920… │ True   │
│  1.87911862e-12]                                              │                    │         │         │        │
│ [ 4.26829265 10.          0.          4.          0.        ] │ 1479.9756074274385 │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26829262 10.          0.          4.          0.        ] │ 1479.9756057940485 │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26829255 10.          0.          4.          0.        ] │ 1479.9756005180443 │ 4243.9… │ 0.2920… │ True   │
│ [4.26829183e+00 1.00000000e+01 5.35694927e-11 4.00000000e+00  │ 1479.975552510574  │ 4243.9… │ 0.2920… │ True   │
│  0.00000000e+00]                                              │                    │         │         │        │
│ [ 4.26829159 10.          0.          4.          0.        ] │ 1479.9755365701533 │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26828415 10.          0.          4.          0.        ] │ 1479.9750381427898 │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26823665 10.          0.          4.          0.        ] │ 1479.9718553140679 │ 4243.9… │ 0.2920… │ True   │
│ [4.26829268e+00 1.00000000e+01 0.00000000e+00 4.00000000e+00  │ 1479.982273306839  │ 4244.0… │ 0.2920… │ True   │
│  2.89719599e-04]                                              │                    │         │         │        │
│ [ 4.26817043 10.          0.          4.          0.        ] │ 1479.967418680113  │ 4243.9… │ 0.2920… │ True   │
│ [ 4.26793704 10.          0.          4.          0.        ] │ 1479.951781715626  │ 4243.9… │ 0.2920… │ True   │
│ [4.26645196e+00 1.00000000e+01 3.72767516e-03 3.99958174e+00  │ 1479.9828745921043 │ 4243.8… │ 0.2921… │ True   │
│  0.00000000e+00]                                              │                    │         │         │        │
│ [ 4.26400874 10.          0.          4.          0.        ] │ 1479.688585487611  │ 4243.2… │ 0.2921… │ True   │
│ [4.26829216e+00 9.99831510e+00 4.41716029e-05 3.99998523e+00  │ 1479.8230990158304 │ 4243.4… │ 0.2921… │ True   │
│  0.00000000e+00]                                      